In [ ]:
import pathlib, os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['HF_HOME'] = str(pathlib.Path("~/scratch-llm/storage/cache/huggingface/").expanduser().absolute()) # '/scratch-llm/storage/cache/'
# os.environ["TRANSFORMERS_CACHE"] = "~/scratch-llm/storage/models/"

import torch, pickle
import numpy as np

from transformers import AutoTokenizer
from nebulagraph_lite import nebulagraph_let as ng_let
from llama_index.graph_stores.nebula import NebulaPropertyGraphStore

from llama_index.core import Settings
from llama_index.core.schema import TextNode
from llama_index.core.prompts import PromptTemplate
from llama_index.core.response_synthesizers import TreeSummarize
from llama_index.core.vector_stores.simple import SimpleVectorStoreData, SimpleVectorStore, VectorStoreQuery

from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

from typing import List
from numpy import dot
from numpy.linalg import norm
from pydantic import BaseModel, Field
from llama_index.core.output_parsers import PydanticOutputParser

# NebulaGraph conexion

In [ ]:
# load NebulaGraph JupyterNotebook extension
# !udocker pull vesoft/nebula-metad:v3
# !udocker create --name=nebula-metad vesoft/nebula-metad:v3
# !udocker setup --execmode=F1 nebula-metad
# !udocker pull vesoft/nebula-graphd:v3
# !udocker create --name=nebula-graphd vesoft/nebula-graphd:v3
# !udocker setup --execmode=F1 nebula-graphd
# !udocker pull vesoft/nebula-storaged:v3
# !udocker create --name=nebula-storaged vesoft/nebula-storaged:v3
# !udocker setup --execmode=F1 nebula-storaged

n = ng_let(in_container=True)
n.start() # Takes around 5 mins

Info: downloading layer sha256:20717c72714566f0a51c0f26f7237ae23515d730d9ffe7b8c34bb0f448d23beb
Info: downloading layer sha256:f73b0b86b2168fee37455b76bbb9a850d41ad48dda1544fa5a4c8fb933bfa006
Info: downloading layer sha256:4f4fb700ef54461cfa02571ae0db9a0dc1e0cdb5577484a6d75e68dc38e8acc1
Info: downloading layer sha256:3567ff3965abe5d66952e4a8be22af4c6e9a95d2ac02ada22ac6c0b0be37a2c1
Info: downloading layer sha256:5beb905c3081ead3dd9f9a60673fd722da67b135bc558bc71b92d69ec12ae999
Info: downloading layer sha256:2d473b07cdd5f0912cd6f1a703352c82b512407db6b05b43f2553732b55df3bc
Error: container name already exists
Info: downloading layer sha256:1ac1fd25afe5d455d2582e90b07d4eedabf5eae8e66b6508893bb030f7db5157
Info: downloading layer sha256:ca0b24da507dc453e8724ea7a219ee899c2f3020d2828e982d0e36bf546ba2f4
Info: downloading layer sha256:4f4fb700ef54461cfa02571ae0db9a0dc1e0cdb5577484a6d75e68dc38e8acc1
Info: downloading layer sha256:b3d21254eb5aefb20a7ac54f1000092a19fd6a4a874319ccae2ccbed0ee5af6e
Inf

In [4]:
%reload_ext ngql
%ngql --address 127.0.0.1 --port 9669 --user root --password nebula

[OK] Connection Pool Created


,Name
0,PrimeKG
1,PrimeKG_nebula
2,basketballplayer


# Vector + Graph store

## SimpleVectorStore:

In [5]:
# Load the actual data into all_nodes_embeddded
with open(os.path.expanduser('~/scratch-llm/storage/nodes/all_nodes_all-mpnet-base-v2.pkl'), 'rb') as f:
    all_nodes_embedded: List[TextNode] = pickle.load(f)
# Create dictionaries from the nodes
embedding_dict = {node.id_: node.get_embedding() for node in all_nodes_embedded}
text_id_to_ref_doc_id = {node.id_: node.ref_doc_id or "None" for node in all_nodes_embedded}
metadata_dict = {node.id_: node.metadata for node in all_nodes_embedded}

# Initialize the SimpleVectorStore with the dictionaries
vector_store = SimpleVectorStore(
    data = SimpleVectorStoreData(
        embedding_dict=embedding_dict,
        text_id_to_ref_doc_id=text_id_to_ref_doc_id,
        metadata_dict=metadata_dict,
    ),
    stores_text=True
)

## NebulaPropertyGraphStore:

In [6]:
graph_store = NebulaPropertyGraphStore(
    space = "PrimeKG",
    username = "root",
    password = "nebula",
    url = "nebula://localhost:9669",
    props_schema= """`node_index` STRING, `node_type` STRING, `node_id` STRING, `node_name` STRING, 
        `node_source` STRING, `mondo_id` STRING, `mondo_name` STRING, `group_id_bert` STRING, 
        `group_name_bert` STRING, `orphanet_prevalence` STRING, `display_relation` STRING """,
)

# LLM

## SciBERT

In [7]:
# tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_cased", padding_side="left", device_map="auto")    
# if tokenizer.pad_token_id is None: #no <pad> token previously defined, only eos_token
#     tokenizer.pad_token = "<|end_of_text|>"
#     tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)

# if tokenizer.eos_token_id is None: # no <eos> token previously defined
#     tokenizer.eos_token = "<|end_of_text|>"
#     tokenizer.eos_token_id = tokenizer.convert_tokens_to_ids(tokenizer.eos_token)


# llm = HuggingFaceLLM(
#     model_name="allenai/scibert_scivocab_cased",
#     context_window=512,
#     max_new_tokens=1048,
#     generate_kwargs={
#         "temperature": 0.10, 
#         "do_sample": True,
#         "pad_token_id": tokenizer.pad_token_id,
#         "top_k": 5, 
#         "top_p": 0.85
#     },
#     model_kwargs={
#         "torch_dtype": torch.float16,
#     },
#     tokenizer=tokenizer,
#     # device_map="auto",  # Automatically offload layers to CPU if GPU memory is insufficient
#     device_map="cuda" if torch.cuda.is_available() else "cpu",
#     stopping_ids=[tokenizer.eos_token_id],
#     tokenizer_kwargs={"max_length": None},
#     is_chat_model=True,
# )

# Settings.llm = llm
# Settings.chunk_size = 1024
# Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2") # BAAI/bge-small-en-v1.5 /  m3

## Llama-3.2-3B-Instruct

In [8]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct", padding_side="left", device_map="auto")    
if tokenizer.pad_token_id is None: #no <pad> token previously defined, only eos_token
    tokenizer.pad_token = "<|end_of_text|>"
    tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)


llm = HuggingFaceLLM(
    model_name="meta-llama/Llama-3.2-3B-Instruct",
    context_window=8192,
    max_new_tokens=3048,
    generate_kwargs={
        "temperature": 0.10, 
        "do_sample": True,
        "pad_token_id": tokenizer.pad_token_id,
        "top_k": 10, 
        "top_p": 0.9,
        # "repetition_penalty": 0.9,  # Added to reduce repetition
        # "no_repeat_ngram_size": 3,  # Prevents repetition of n-grams
    },
    model_kwargs={
        "torch_dtype": torch.float16,
    },
    tokenizer=tokenizer,
    # device_map="auto",  # Automatically offload layers to CPU if GPU memory is insufficient
    device_map="cuda" if torch.cuda.is_available() else "cpu",
    stopping_ids=[tokenizer.eos_token_id],
    tokenizer_kwargs={"max_length": None},
    is_chat_model=True,
)

Settings.llm = llm
Settings.chunk_size = 1024
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2") # BAAI/bge-small-en-v1.5 /  m3 / sentence-transformers/all-mpnet-base-v2

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Llama-3.3-70B-Instruct

In [9]:
# # for Llama-3.3-70B-Instruct
# from transformers import BitsAndBytesConfig
# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.float16,
#     bnb_4bit_quant_type="nf4"
# )

# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.3-70B-Instruct", padding_side="left", device_map="auto")    
# if tokenizer.pad_token_id is None: #no <pad> token previously defined, only eos_token
#     tokenizer.pad_token = "<|end_of_text|>"
#     tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)


# llm = HuggingFaceLLM(
#     model_name="meta-llama/Llama-3.3-70B-Instruct",
#     context_window=1048,
#     max_new_tokens=256,
#     generate_kwargs={
#         "temperature": 0.10, 
#         "do_sample": True,
#         "pad_token_id": tokenizer.pad_token_id,
#         "top_k": 5, 
#         "top_p": 0.85
#     },
#     model_kwargs={
#         "torch_dtype": torch.float16,
#         "quantization_config": quantization_config,
#     },
#     tokenizer=tokenizer,
#     # device_map="auto",  # Automatically offload layers to CPU if GPU memory is insufficient
#     device_map="cuda" if torch.cuda.is_available() else "cpu",
#     stopping_ids=[tokenizer.eos_token_id],
#     tokenizer_kwargs={"max_length": None},
#     is_chat_model=True,
# )

# Settings.llm = llm
# Settings.chunk_size = 1024
# Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5") # BAAI/bge-small-en-v1.5 /  m3

# DiseaseMode

MetadataFilters:

In [10]:
from llama_index.core.vector_stores.types import MetadataFilters, FilterOperator
disease_dict = {
    "key": "node_type",
    "value": "disease",
    "operator": FilterOperator.EQ
}
disease_filter = MetadataFilters(filters=[disease_dict])

In [11]:
class DiseaseMode():
    def __init__(self, vector_store: SimpleVectorStore, graph_store: NebulaPropertyGraphStore):
        self.vector_store = vector_store
        self.graph_store = graph_store

    def retrieve(self, query: str):        
        query_embedding = Settings.embed_model.get_text_embedding(query)
        vector_results = self.vector_store.query(
            VectorStoreQuery(
                query_embedding=query_embedding, 
                similarity_top_k=1,
                filters=disease_filter,
            )
        )
        
        top_node_id = vector_results.ids[0]
        top_node_score = vector_results.similarities[0]
        kg_node = self.graph_store.get(ids=[top_node_id])[0]
                
        results = [{ # Create results list with primary node
            "node_index": kg_node.id_,
            "node_name": kg_node.properties["node_name"],
            "text": getattr(kg_node, "text", ""),
            "score": top_node_score
        }]
        
        print(f"Best node from vector query: Node ID: {kg_node.id_}, "
              f"Score: {top_node_score:.4f}, Name: {kg_node.properties['node_name']}")
        
        # Find related nodes through graph query
        graph_nodes = self.graph_store.structured_query(
            """
            MATCH (e:Node__) WHERE id(e) == $ids
            MATCH p=(e)-[r:Relation__{label:"disease-disease"}]-(t) 
            UNWIND relationships(p) as rel
            RETURN DISTINCT id(t), t.Props__.node_name, t.Chunk__.text
            """, 
            param_map={"ids": top_node_id}
        )
        
        # Calculate similarity for related nodes and add relevant ones to results
        all_similarities = []
        for node in graph_nodes:
            node_text = node["t.Props__.node_name"] + ": " + node["t.Chunk__.text"]
            node_embedding = Settings.embed_model.get_text_embedding(node_text)
                    
            similarity = dot(query_embedding, node_embedding) / (norm(query_embedding) * norm(node_embedding))
            all_similarities.append((node, similarity))
            
        if len(all_similarities) > 3:
            sim = [s for _, s in all_similarities]
            threshold = np.percentile(sim, 75) # keep top 25% of nodes
        else:
            threshold = 0.7
        
        for node, similarity in all_similarities:
            if similarity > threshold: #and similarity >= 0.5:
                results.append({
                    "node_index": node["id(t)"],
                    "node_name": node["t.Props__.node_name"],
                    "text": node_text,
                    "score": similarity
                })
        
        results = sorted(results, key=lambda x: x["score"], reverse=True)
        print("\nBest related nodes from graph query:")
        for node in results:  # Skip primary node
            print(f"ID: {node['node_index']} | node name: {node['node_name']} | score: {node['score']:.4f}")
        
        graph_phenotype = graph_store.structured_query(
            """
            MATCH (e:Node__) WHERE id(e) == $ids
            MATCH (e)-[r:Relation__{label:"disease-phenotype-positive"}]-(t) 
            RETURN DISTINCT id(t), t.Props__.node_name
            """, 
            param_map={"ids": top_node_id}
        )
        # join the phenotype names without the " '' " characters
        phenotypes = ", ".join(node["t.Props__.node_name"].replace("'", "") for node in graph_phenotype)

        nodes_with_text = [node for node in results if node['text'].strip()]
        context = [f"'{node['node_name']}': {node['text']}" for node in nodes_with_text] if nodes_with_text else None
        phenotype_context = [f"Is associated with the following phenotypes: {phenotypes}\n"] if phenotypes else None
        
        if results:
            return (context, phenotype_context)
        else:
            return [f"No graph relationships found for {results[0]['node_name']}"] if results else ["No results found"]


## Prompt templates:

In [83]:
context_phenotype_template = """
    I need information about {query_str}.
    
    Context information is below:
    {text_chunks}

    Phenotype context is below:
    {phenotype_context}

    You are a medical knowledge assistant specializing in rare diseases. Your task is to create a comprehensive list of symptoms for {query_str}.

    CRITICAL INSTRUCTIONS:
    1. Extract ONLY symptoms mentioned in the context and/or phenotype context
    2. Return EXACTLY the 15 most relevant symptoms
    3. Use HPO medical terminology
    4. Return EXACTLY this JSON format (no variations):

    Always format your response as a VALID JSON:
    {
        "disease": "{query_str}",
        "symptoms": [
            "symptom1: Brief explanation using HPO nomenclature.",
            "symptom2: Brief explanation using HPO nomenclature.",
        ]
    }

    Do NOT use nested objects. Use exactly "disease" and "symptoms" as shown.
"""

In [103]:
no_rag_template = """
    I need information about {query_str}.

    You are a medical knowledge assistant specializing in rare diseases. Your task is to create a comprehensive list of symptoms for {query_str}.

    CRITICAL INSTRUCTIONS:
    1. Return EXACTLY the 15 most relevant symptoms
    2. Use HPO medical terminology
    3. Return EXACTLY this JSON format (no variations):

    Always format your response as a VALID JSON:
    {
        "disease": "{query_str}",
        "symptoms": [
            "symptom1: Brief explanation using HPO nomenclature.",
            "symptom2: Brief explanation using HPO nomenclature.",
        ]
    }

    Do NOT use nested objects. Use exactly "disease" and "symptoms" as shown.
"""


## Chat

In [ ]:
user = "reproductive system cancer" # "autosomal recessive polycystic kidney disease"
context, phenotypes = DiseaseMode(vector_store, graph_store).retrieve(user)

class Output(BaseModel):
    disease: str = Field(
        description="The disease of interest given by the user."
    )
    symptoms: List[str] = Field(
        description="Symptoms associated with the disease."
    )
output_parser = PydanticOutputParser(Output)

if context and phenotypes:
    print(f"Context: {context[:100]}")
    print(f"Phenotypes: {phenotypes}\n")
    prompt_template = PromptTemplate(context_phenotype_template) 
    summarizer = TreeSummarize(verbose=True, llm=llm, summary_template=prompt_template)

    response = summarizer.get_response(
        query_str = user,
        text_chunks = "\n".join(context),
        phenotype_context = phenotypes
    )

elif context and phenotypes == None: # No phenotype edges found
    print(f"Context: {context[:100]}")
    print(f"Phenotypes: {phenotypes}\n")
    prompt_template = PromptTemplate(context_phenotype_template)
    summarizer = TreeSummarize(verbose=True, llm=llm, summary_template=prompt_template)

    response = summarizer.get_response(
        query_str = user,
        text_chunks = "\n".join(context),
        phenotype_context = ""
    )
    
elif context == None and phenotypes:
    print(f"Context: {context}")
    print(f"Phenotypes: {phenotypes}\n")
    prompt_template = PromptTemplate(context_phenotype_template)
    summarizer = TreeSummarize(verbose=True, llm=llm, summary_template=prompt_template)

    response = summarizer.get_response(
        query_str = user,
        text_chunks = "",  # No context available
        phenotype_context = phenotypes
    )

elif context == None and phenotypes == None:
    print(f"No context or phenotypes found for {user}")
    response = "No context or phenotypes found for the given disease."


try:
    parsed_response = output_parser.parse(response)
    print("\nRESPONSE OK:", parsed_response.model_dump_json())
    print(f"Symptoms retrieved: {len(parsed_response.symptoms)} for {parsed_response.disease}")
except ValueError as e:
    print(f"Warning: {e}")
    # Fallback: create an Output object with empty symptoms and the disease name
    parsed_response = Output(disease=user, symptoms=[])
    print("\n", parsed_response.model_dump_json())
    print(f"{len(parsed_response.symptoms)} symptoms for {parsed_response.disease}")

Best node from vector query: Node ID: 35443, Score: 0.6819, Name: reproductive system disease

Best related nodes from graph query:
ID: 35443 | node name: reproductive system disease | score: 0.6819
ID: 35840 | node name: female reproductive system disease | score: 0.6610
ID: 35622 | node name: reproductive system neoplasm | score: 0.6406
ID: 35775 | node name: male reproductive system disease | score: 0.6024
Context: ["'reproductive system disease': A disease involving the reproductive system. A non-neoplastic disorder that affects the male or female reproductive system. Representative examples include prostatitis, balanitis, phimosis, endometriosis, and pelvic inflammatory disease.; deviation from or interruption of the normal structure or function of the reproductive system.; A non-neoplastic or neoplastic disorder that affects the male or female genital system. Representative examples of non-neoplastic disorders include infection, testicular torsion, endometriosis, and adenomyosis.

In [105]:
from llama_index.core.base.llms.types import ChatMessage
# include a prompt template for the LLM chat 
template = PromptTemplate(no_rag_template)
prompt = template.format(query_str=user)
response = llm.chat([ChatMessage(role="user", content=prompt)])
print("\nLLM Response:", response)


LLM Response: assistant: Here is the list of symptoms for reproductive system cancer in the required format:

{
    "disease": "reproductive system cancer",
    "symptoms": [
        "Abnormal vaginal bleeding: Unusual or persistent bleeding from the vagina, which may be accompanied by clots or discharge.",
        "Abnormal uterine bleeding: Unusual or irregular bleeding from the uterus, which may be accompanied by clots or discharge.",
        "Abnormal cervical bleeding: Unusual or irregular bleeding from the cervix, which may be accompanied by clots or discharge.",
        "Pelvic pain: Pain in the lower abdomen or pelvis, which may be constant or intermittent.",
        "Pelvic pressure: Feeling of pressure or fullness in the lower abdomen or pelvis.",
        "Abdominal swelling: Swelling or enlargement of the abdomen, which may be accompanied by discomfort or pain.",
        "Weight loss: Unintentional weight loss, which may be accompanied by fatigue or weakness.",
        "Fat

#### Checks with NebulaGraph 

In [14]:
%ngql CREATE SPACE IF NOT EXISTS PrimeKG(vid_type=FIXED_STRING(256))
%ngql USE PrimeKG;

""


In [ ]:
%%ngql
MATCH (e:Node__)
WHERE id(e) == "38849"
RETURN DISTINCT 
        e.Props__.node_name, 
        e.Chunk__.text

,e.Props__.node_name,e.Chunk__.text
0,Wieacker-Wolff syndrome (spectrum),


In [43]:
%%ngql
# find all disease nodes where e.Chunk__.text is empty
MATCH (e:Node__)
MATCH (e)-[r1:Relation__{label:"disease-phenotype-positive"}]-(t1) # {label:"disease-phenotype-positive"}
WHERE e.Props__.node_type == "disease" AND e.Chunk__.text == ""
RETURN DISTINCT 
    e.Props__.node_name,
    # e.Props__.node_type,
    e.Chunk__.text
    # t1.Props__.node_name

,e.Props__.node_name,e.Chunk__.text
0,familial focal alopecia,
1,myelolymphatic insufficiency,
2,amyotrophic dystonic paraplegia,
3,symphalangism of toes,
4,"cerebellar, ocular, craniofacial, and genital syndrome",
...,...,...
1098,neurodevelopmental disorder with relative macrocephaly and with or without cardiac or endocrine anomalies,
1099,"obsolete short sleep, familial natural",
1100,"antithrombin, familial hemorrhagic diathesis due to",
1101,optic atrophy with negative Electroretinograms,


In [ ]:
# %%ngql
# MATCH (e:Node__)
# WHERE id(e) == "27781"
# MATCH (e)-[r1:Relation__{label:"disease-phenotype-positive"}]-(t1) # {label:"disease-phenotype-positive"}
# # MATCH (e)-[r2:Relation__{label:"disease-disease"}]-(t2)
# # RETURN e.Props__.node_name, e.Chunk__.text

# RETURN DISTINCT 
#         e.Props__.node_name, 
#         # e.Chunk__.text
#         r1.label,
#         id(t1),
#         t1.Props__.node_name
#         # r2.label,
#         # id(t2) AS index_disease,
#         # t2.Props__.node_name AS name_disease
#         # t.Chunk__.text


In [ ]:
n.stop()